In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("MiAplicacion").getOrCreate()

25/03/27 10:43:46 WARN Utils: Your hostname, mauro-desktop resolves to a loopback address: 127.0.1.1; using 192.168.0.21 instead (on interface wlxf4ec3892e485)
25/03/27 10:43:46 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/27 10:43:46 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


### Propiedad Descripción

**event_time** - Hora en la que ocurrió el evento (en UTC).

**event_type** - Solo un tipo de evento: view - cart - remove_from_cart - purchase.

**product_id** - ID de un producto.

**category_id** - ID de la categoría del producto.

**category_code** - Taxonomía de la categoría del producto (nombre en código) si fue posible obtenerla. Usualmente presente para categorías significativas y omitida para diferentes tipos de accesorios.

**brand** - Cadena de texto en minúsculas del nombre de la marca. Puede faltar.

**price** - Precio flotante de un producto. Presente.

**user_id** - ID de usuario permanente

**user_session** - ID de sesión temporal del usuario. Es el mismo para cada sesión del usuario. Cambia cada vez que el usuario regresa a la tienda online después de una pausa larga.

In [3]:
from pyspark.sql import functions as F

In [4]:
df = spark.read.options( header = 'True', inferSchema = 'True' ).csv("archive/*.csv")

In [6]:
df.show()

+-------------------+----------------+----------+-------------------+-------------+---------+-----+---------+--------------------+
|         event_time|      event_type|product_id|        category_id|category_code|    brand|price|  user_id|        user_session|
+-------------------+----------------+----------+-------------------+-------------+---------+-----+---------+--------------------+
|2019-11-30 21:00:00|remove_from_cart|   5712790|1487580005268456287|         NULL|    f.o.x| 6.27|576802932|51d85cb0-897f-48d...|
|2019-11-30 21:00:00|            view|   5764655|1487580005411062629|         NULL|      cnd|29.05|412120092|8adff31e-2051-489...|
|2019-11-30 21:00:02|            cart|      4958|1487580009471148064|         NULL|   runail| 1.19|494077766|c99a50e8-2fac-4c4...|
|2019-11-30 21:00:05|            view|   5848413|1487580007675986893|         NULL|freedecor| 0.79|348405118|722ffea5-73c0-492...|
|2019-11-30 21:00:07|            view|   5824148|1487580005511725929|         NULL|

In [ ]:
df.filter(F.col("price") < 0).show()

In [8]:
df = df.withColumn("price",F.when(F.col("price") < 0, F.abs(F.col("price"))).otherwise(F.col("price")))

In [9]:
df.filter(F.col("price") < 0).show()

+----------+----------+----------+-----------+-------------+-----+-----+-------+------------+
|event_time|event_type|product_id|category_id|category_code|brand|price|user_id|user_session|
+----------+----------+----------+-----------+-------------+-----+-----+-------+------------+
+----------+----------+----------+-----------+-------------+-----+-----+-------+------------+



Tasa de conversión por producto: ¿Cuántos productos que fueron vistos terminaron en compra?

In [10]:
conversion_reates = df.groupBy("product_id").agg(
    F.sum(F.when(F.col("event_type") == "purchase",1).otherwise(0)).alias("purchases"),
    F.sum(F.when(F.col("event_type") == "view",1).otherwise(0)).alias("views")
).withColumn("conversion_rate", F.col("purchases") / F.col("views"))

In [11]:
conversion_reates.show()

+----------+---------+-----+--------------------+
|product_id|purchases|views|     conversion_rate|
+----------+---------+-----+--------------------+
|   5819251|       40|  261|  0.1532567049808429|
|   5657304|       12|  571|0.021015761821366025|
|   5815137|       38|  236| 0.16101694915254236|
|   5809871|     1100| 3868|  0.2843846949327818|
|   5812121|      108|  781|  0.1382842509603073|
|   5834157|      126| 1689| 0.07460035523978685|
|   5908237|       20|  100|                 0.2|
|   5826154|       96|  697| 0.13773314203730272|
|   5670719|       69|  114|  0.6052631578947368|
|   5767927|      299|  814| 0.36732186732186733|
|   5574996|       19| 1290|0.014728682170542635|
|   5900643|       58|  331| 0.17522658610271905|
|   5876926|       94| 2055|0.045742092457420926|
|   5863096|      107|  181|  0.5911602209944752|
|   5713340|       15|  110| 0.13636363636363635|
|   5557784|        8|  154| 0.05194805194805195|
|   5797144|       19|  187| 0.10160427807486631|


Tiempo hasta la compra ¿Cuánto tiempo tarda un usuario en comprar desde su primer evento de vista?

In [16]:
df_filtered = df.filter(F.col("event_type").isin(["view", "purchase"]))

purchase_time = df_filtered.groupBy("user_session","product_id").agg(
    F.min(F.when(F.col("event_type") == "view",F.col("event_time"))).alias("first_view"),
    F.min(F.when(F.col("event_type") == "purchase",F.col("event_time"))).alias("first_purchase")
).withColumn("time_to_purchase", F.col("first_purchase").cast("long") - F.col("first_view").cast("long"))

In [19]:
purchase_time.filter(F.col("time_to_purchase").isNotNull()).show()

+--------------------+----------+-------------------+-------------------+----------------+
|        user_session|product_id|         first_view|     first_purchase|time_to_purchase|
+--------------------+----------+-------------------+-------------------+----------------+
|885948cc-afa3-46a...|   5866173|2019-10-20 07:39:48|2019-10-20 04:47:40|          -10328|
|8e64c2f8-0104-44e...|   5900297|2019-12-01 03:31:27|2019-12-01 04:35:08|            3821|
|77e818ab-4d9f-4a1...|      4938|2019-12-01 11:24:41|2019-12-01 11:45:48|            1267|
|00e83d38-8c67-4e5...|   5833332|2019-12-01 11:58:48|2019-12-01 12:17:09|            1101|
|8ed1031c-601e-4c8...|   5819270|2019-12-01 12:29:21|2019-12-01 12:42:24|             783|
|a9f51db0-7030-4d5...|   5699414|2019-12-01 13:01:56|2019-12-01 13:38:06|            2170|
|5895cc10-6951-446...|   5787014|2019-12-01 14:32:50|2019-12-01 14:52:21|            1171|
|f8499285-f42b-437...|   5862936|2019-12-01 14:45:35|2019-12-01 15:31:18|            2743|

Algunas dudas sobre este punto. Porque hay negativos? Que pasa si miro un producto luego de comprarlo?

Frecuencia de compra por usuario ¿Qué tan leales son los usuarios? ¿Compran repetidamente?

In [23]:
purchase_freq = df.filter(F.col("event_type") == "purchase") \
                    .groupBy("user_id") \
                    .agg(F.count("*").alias("total_purchases"))

In [25]:
purchase_freq.show()

+---------+---------------+
|  user_id|total_purchases|
+---------+---------------+
|578779619|              9|
|580197384|              8|
|479913443|             63|
|579543348|              9|
|539938733|             22|
|569691461|              7|
|522385131|            100|
|461446900|             19|
|445185747|             13|
|398719446|             10|
|543676273|             33|
|580799959|             13|
|581227067|              8|
|568429315|              6|
|563476291|             20|
|439472531|              6|
|582006763|             29|
|582162065|              2|
|582303660|             12|
|569728881|              3|
+---------+---------------+
only showing top 20 rows



Segmentación por precios
¿Cuáles son los rangos de precio más populares?

In [10]:
price_distribution = df.filter(F.col("event_type") == "purchase") \
                            .groupBy(F.floor(F.col("price") / 50) * 50) \
                            .agg(F.count("*").alias("count")) \
                            .withColumnRenamed("(floor((price / 50)) * 50)", "price range") \
                            .orderBy("count", ascending = False)   

price_distribution.show() 

+-----------+-------+
|price range|  count|
+-----------+-------+
|          0|1280230|
|         50|   5198|
|        150|    655|
|        100|    646|
|        200|    155|
|        250|    116|
|        300|      7|
+-----------+-------+



Análisis de abandono de carrito ¿Qué productos se agregan al carrito pero no se compran?

In [37]:
cart_vs_purchase = df.groupBy("product_id"). agg(
    F.sum(F.when(F.col("event_type") == "cart", 1).otherwise(0)).alias("added_to_cart"),
    F.sum(F.when(F.col("event_type") == "purchase", 1).otherwise(0)).alias("purchased")


).withColumn("abandon_rate", (F.col("added_to_cart") - F.col("purchased")) / F.col("added_to_cart"))

cart_vs_purchase.show()

+----------+-------------+---------+------------------+
|product_id|added_to_cart|purchased|      abandon_rate|
+----------+-------------+---------+------------------+
|   5819251|          246|       40|0.8373983739837398|
|   5657304|           83|       12|0.8554216867469879|
|   5815137|          153|       38|0.7516339869281046|
|   5809871|         3989|     1100|0.7242416645775883|
|   5812121|          484|      108|0.7768595041322314|
|   5834157|          660|      126|0.8090909090909091|
|   5908237|           95|       20|0.7894736842105263|
|   5826154|          390|       96|0.7538461538461538|
|   5670719|          225|       69|0.6933333333333334|
|   5767927|          809|      299| 0.630407911001236|
|   5574996|          125|       19|             0.848|
|   5900643|          192|       58|0.6979166666666666|
|   5876926|          509|       94|0.8153241650294696|
|   5863096|          637|      107|0.8320251177394035|
|   5713340|           47|       15|0.6808510638